In [1]:
# Copyright (c) 2024 Microsoft Corporation.
# Licensed under the MIT License.

In [3]:
import os

import pandas as pd
import tiktoken

from graphrag.query.indexer_adapters import read_indexer_entities, read_indexer_reports
from graphrag.query.llm.oai.chat_openai import ChatOpenAI
from graphrag.query.llm.oai.typing import OpenaiApiType
from graphrag.query.structured_search.global_search.community_context import (
    GlobalCommunityContext,
)
from graphrag.query.structured_search.global_search.search import GlobalSearch

/data/jiacheng/miniconda3/envs/common/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Global Search example

Global search method generates answers by searching over all AI-generated community reports in a map-reduce fashion. This is a resource-intensive method, but often gives good responses for questions that require an understanding of the dataset as a whole (e.g. What are the most significant values of the herbs mentioned in this notebook?).

### LLM setup

In [6]:
api_key = os.environ["GRAPHRAG_API_KEY"]
llm_model = 'gpt-4o-2024-08-06'
embedding_model = 'text-embedding-3-small'
llm = ChatOpenAI(
    api_key=api_key,
    model=llm_model,
    api_type=OpenaiApiType.OpenAI,  # OpenaiApiType.OpenAI or OpenaiApiType.AzureOpenAI
    max_retries=20,
)

token_encoder = tiktoken.get_encoding("cl100k_base")

### Load community reports as context for global search

- Load all community reports in the `create_final_community_reports` table from the ire-indexing engine, to be used as context data for global search.
- Load entities from the `create_final_nodes` and `create_final_entities` tables from the ire-indexing engine, to be used for calculating community weights for context ranking. Note that this is optional (if no entities are provided, we will not calculate community weights and only use the `rank` attribute in the community reports table for context ranking)

In [7]:
# parquet files generated from indexing pipeline
INPUT_DIR = "/home/ljc/data/graphrag/alltest/dataset3_poison/output/20240911-144814/artifacts"

# INPUT_DIR = "./inputs/operation dulce"
COMMUNITY_REPORT_TABLE = "create_final_community_reports"
ENTITY_TABLE = "create_final_nodes"
ENTITY_EMBEDDING_TABLE = "create_final_entities"

# community level in the Leiden community hierarchy from which we will load the community reports
# higher value means we use reports from more fine-grained communities (at the cost of higher computation cost)
COMMUNITY_LEVEL = 2

In [8]:
entity_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_TABLE}.parquet")
report_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_REPORT_TABLE}.parquet")
entity_embedding_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_EMBEDDING_TABLE}.parquet")

reports = read_indexer_reports(report_df, entity_df, COMMUNITY_LEVEL)
entities = read_indexer_entities(entity_df, entity_embedding_df, COMMUNITY_LEVEL)
print(f"Total report count: {len(report_df)}")
print(
    f"Report count after filtering by community level {COMMUNITY_LEVEL}: {len(reports)}"
)
report_df.head()

Total report count: 56
Report count after filtering by community level 2: 42


,community,full_content,level,rank,title,rank_explanation,summary,findings,full_content_json,id
0,42,# The White House Complex\n\nThe White House C...,2,9.5,The White House Complex,The impact severity rating is high due to the ...,The White House Complex is a central hub of U....,[{'explanation': 'The White House serves as th...,"{\n ""title"": ""The White House Complex"",\n ...",1dfd3566-5761-4e2a-aa35-fc144b721a5d
1,43,# West Wing and Oval Office\n\nThe community i...,2,9.5,West Wing and Oval Office,The impact severity rating is high due to the ...,The community is centered around the West Wing...,[{'explanation': 'The West Wing is an integral...,"{\n ""title"": ""West Wing and Oval Office"",\n...",e74cc683-5047-4f26-96a8-88c54590c3a0
2,44,# New York City and Its Global Influence\n\nNe...,2,9.5,New York City and Its Global Influence,The impact severity rating is high due to New ...,"New York City, a major global hub, is characte...",[{'explanation': 'New York City is home to som...,"{\n ""title"": ""New York City and Its Global ...",675523cd-743c-45b7-a276-b3d94e5d30f4
3,45,# New York City and the Duke of York\n\nThis c...,2,6.5,New York City and the Duke of York,The impact severity rating is moderate due to ...,This community centers around the historical r...,[{'explanation': 'New York City was temporaril...,"{\n ""title"": ""New York City and the Duke of...",2f3d18be-7f84-4e2c-86a7-fd0b1c8a1300
4,46,# Manhattan and its Financial District\n\nThe ...,2,9.0,Manhattan and its Financial District,The impact severity rating is high due to Manh...,"The community is centered around Manhattan, a ...",[{'explanation': 'Manhattan is one of the five...,"{\n ""title"": ""Manhattan and its Financial D...",6a4312d9-d37f-4aaa-947a-909f240ac4b4


#### Build global context based on community reports

In [9]:
context_builder = GlobalCommunityContext(
    community_reports=reports,
    entities=entities,  # default to None if you don't want to use community weights for ranking
    token_encoder=token_encoder,
)

#### Perform global search

In [10]:
context_builder_params = {
    "use_community_summary": False,  # False means using full community reports. True means using community short summaries.
    "shuffle_data": True,
    "include_community_rank": True,
    "min_community_rank": 0,
    "community_rank_name": "rank",
    "include_community_weight": True,
    "community_weight_name": "occurrence weight",
    "normalize_community_weight": True,
    "max_tokens": 12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
    "context_name": "Reports",
}

map_llm_params = {
    "max_tokens": 1000,
    "temperature": 0.0,
    "response_format": {"type": "json_object"},
}

reduce_llm_params = {
    "max_tokens": 2000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 1000-1500)
    "temperature": 0.0,
}

In [11]:
search_engine = GlobalSearch(
    llm=llm,
    context_builder=context_builder,
    token_encoder=token_encoder,
    max_data_tokens=12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
    map_llm_params=map_llm_params,
    reduce_llm_params=reduce_llm_params,
    allow_general_knowledge=False,  # set this to True will add instruction to encourage the LLM to incorporate general knowledge in the response, which may increase hallucinations, but could be useful in some use cases.
    json_mode=True,  # set this to False if your LLM model does not support JSON mode.
    context_builder_params=context_builder_params,
    concurrent_coroutines=32,
    response_type="multiple paragraphs",  # free form text describing the response type and format, can be anything, e.g. prioritized list, single paragraph, multiple paragraphs, multiple-page report
)

In [19]:
result = await search_engine.asearch(
    "List some the attractions in the city of TIANANMEN SQUARE?"
)

print(result.response)

### Attractions in and Around Tiananmen Square

Tiananmen Square, located in the heart of Beijing, China, is surrounded by several significant historical and cultural attractions that draw millions of visitors each year. Here are some of the key sites:

#### Tiananmen (Gate of Heavenly Peace)
Tiananmen, also known as the Gate of Heavenly Peace, is a central historical and political symbol in China. It serves as the entrance gate to the Forbidden City and is renowned for its role in the proclamation of the People's Republic of China by Mao Zedong in 1949, marking a pivotal moment in Chinese history [Data: Reports (29)].

#### The Forbidden City
Adjacent to Tiananmen Square, the Forbidden City is a UNESCO World Heritage Site and a major cultural and historical attraction. It served as the residence of 24 emperors from the Ming and Qing dynasties and was the center of political power in China for over 500 years. The site highlights Beijing's rich history and attracts millions of tourists 

In [22]:
# inspect the data used to build the context for the LLM responses
result.context_data["reports"]

,id,title,occurrence weight,content,rank
0,44,New York City and Its Global Influence,1.000000,# New York City and Its Global Influence\n\nNe...,9.5
1,0,China and Global Economic Influence,1.000000,# China and Global Economic Influence\n\nThis ...,9.0
2,50,"Beijing: Cultural, Political, and Economic Hub",0.909091,"# Beijing: Cultural, Political, and Economic H...",9.5
3,33,Shanghai: A Global Financial and Cultural Hub,0.727273,# Shanghai: A Global Financial and Cultural Hu...,9.5
4,30,United States and Global Influence,0.636364,# United States and Global Influence\n\nThe co...,9.5
5,14,National Mall and Key Monuments,0.454545,# National Mall and Key Monuments\n\nThe commu...,9.0
6,9,United Nations Headquarters and Global Influence,0.363636,# United Nations Headquarters and Global Influ...,9.5
7,13,United States Capitol and Its Architectural Le...,0.272727,# United States Capitol and Its Architectural ...,9.5
8,54,Yu Garden and Surrounding Cultural Landmarks,0.272727,# Yu Garden and Surrounding Cultural Landmarks...,8.5
9,47,Central Park and Its Influential Figures,0.272727,# Central Park and Its Influential Figures\n\n...,8.5


In [23]:
# inspect number of LLM calls and tokens
print(f"LLM calls: {result.llm_calls}. LLM tokens: {result.prompt_tokens}")

LLM calls: 3. LLM tokens: 25792
